<a href="https://colab.research.google.com/github/eughappy/HomeWork/blob/recommend_systems/hw10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Встановлюємо деякі бібліотеки для роботи з ДЗ.

In [1]:
!pip install patool
!pip install scikit-surprise

Імпорт потрібних модулів

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import patoolib
from surprise import Dataset, Reader, SVD, NMF, SVDpp
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV

Розпаковуємо архів

In [4]:
patoolib.extract_archive('ml-latest-small.zip')

INFO patool: Extracting ml-latest-small.zip ...
INFO:patool:Extracting ml-latest-small.zip ...
INFO patool: running /usr/bin/7z x -o./Unpack_spvplqk6 -- ml-latest-small.zip
INFO:patool:running /usr/bin/7z x -o./Unpack_spvplqk6 -- ml-latest-small.zip
INFO patool:     with input=''
INFO:patool:    with input=''
INFO patool: ... ml-latest-small.zip extracted to `ml-latest-small2' (local file exists).
INFO:patool:... ml-latest-small.zip extracted to `ml-latest-small2' (local file exists).


'ml-latest-small2'

Створюємо об'єкт рідер та об'єкт датасету, для подальшої роботи з ним

In [5]:
reader = Reader(line_format = 'user item rating' , sep = ',', skip_lines=1, rating_scale = (0.5,5))
ds = Dataset.load_from_file('/content/ml-latest-small/ratings.csv',reader=reader)

Створюємо нашу модель матричної факторизації

In [6]:
svd = SVD()

In [7]:
cross_validate(svd, ds, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8720  0.8682  0.8829  0.8763  0.8689  0.8737  0.0054  
MAE (testset)     0.6708  0.6674  0.6798  0.6706  0.6684  0.6714  0.0044  
Fit time          2.01    2.74    3.14    2.22    1.43    2.31    0.59    
Test time         0.56    0.23    0.47    0.12    0.12    0.30    0.18    


{'test_rmse': array([0.87204234, 0.86817064, 0.8829176 , 0.87626992, 0.8688863 ]),
 'test_mae': array([0.67080479, 0.66741654, 0.67977547, 0.67058332, 0.66843924]),
 'fit_time': (2.0109643936157227,
  2.7440013885498047,
  3.135277509689331,
  2.216446876525879,
  1.4264984130859375),
 'test_time': (0.5633409023284912,
  0.23215627670288086,
  0.4669461250305176,
  0.12061715126037598,
  0.12234020233154297)}

Підбираємо параметри для SVD

In [8]:
params = {'n_factors': [20, 50, 100], 'n_epochs': [5, 10, 20] }
gs = GridSearchCV(SVD, params, measures=['rmse', 'mae'], cv=10)
gs.fit(ds)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8655253156807685
{'n_factors': 20, 'n_epochs': 20}


Cтворюємо NMF модель

In [11]:
nmf = NMF()

In [12]:
cross_validate(nmf, ds, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9252  0.9222  0.9249  0.9172  0.9268  0.9233  0.0034  
MAE (testset)     0.7088  0.7094  0.7043  0.7021  0.7107  0.7071  0.0033  
Fit time          2.57    2.93    3.94    2.56    2.61    2.92    0.53    
Test time         0.23    0.23    0.34    0.11    0.22    0.23    0.07    


{'test_rmse': array([0.92523003, 0.92220984, 0.92494998, 0.91722778, 0.92680469]),
 'test_mae': array([0.70878952, 0.70942533, 0.70426396, 0.70207631, 0.71074319]),
 'fit_time': (2.5664865970611572,
  2.932262420654297,
  3.93656325340271,
  2.5566818714141846,
  2.611300468444824),
 'test_time': (0.2340850830078125,
  0.22584199905395508,
  0.3356931209564209,
  0.10665130615234375,
  0.2237708568572998)}

Підбираємо оптимальні параметри для NMF

In [13]:
params = {'n_factors': [20, 50, 100], 'n_epochs': [5, 10, 20] }
gs = GridSearchCV(NMF, params, measures=['rmse', 'mae'], cv=10)
gs.fit(ds)
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.039377038272806
{'n_factors': 20, 'n_epochs': 20}


**Висновок: Як можна побачити, то SVD має менші RMSE та MAE, що означає, шо SVD краще спрвавляється з передбаченням ніж NMF
найкращі параметри для обох алгоритмів виявилися: {'n_factors': 20, 'n_epochs': 20}**